## Thesis - Maria Julia Luna Gonzalez
### Sector-Based Credit Risk and Stocks Returns
#### Collecting data

In [ ]:
import wrds
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import numpy as np
import plotly.graph_objects as go
import plotly.graph_objects as go

# Connection to WRDS
db = wrds.Connection(wrds_username='')

In [ ]:
# Defining the start_date and end_date

start_date = '1987-05-01'
end_date = '2017-02-28'

# Getting the Credit Risks and the Sectors

query = f'''
    SELECT 
    datadate,
    gvkey,
    splticrm
    FROM comp_na_daily_all.adsprate
    WHERE datadate BETWEEN '{start_date}' AND '{end_date}'
    AND splticrm IS NOT NULL
'''

data = db.raw_sql(query)
df_GSIC = db.get_table(library="comp", table="company", columns =['gvkey','gsector'])
data = data.merge(df_GSIC, on='gvkey', how='left')
data = data.dropna()

In [ ]:
# Getting the Monthly Returns
query2 = f'''
    SELECT 
    datadate,
    gvkey,
    trt1m,
    fic,
    curcdm
    FROM comp_na_daily_all.secm
    WHERE datadate BETWEEN '{start_date}' AND '{end_date}'
    AND fic = 'USA'
    AND curcdm = 'USD'
'''

data2 = db.raw_sql(query2)

merged_data = pd.merge(data, data2, on=['datadate', 'gvkey'], how='left')
merged_data = merged_data.dropna()
merged_data['trt1m'] = merged_data['trt1m'] / 100


In [ ]:
# Getting the FF3 variables 

query3 = f'''
    SELECT 
    dateff,
    smb,
    hml,
    mktrf,
    rf
    FROM ff_all.factors_monthly
    WHERE dateff BETWEEN '{start_date}' AND '{end_date}'
'''

data3 = db.raw_sql(query3)
data3['dateff'] = pd.to_datetime(data3['dateff'])
data3['dateff'] = data3['dateff'] + pd.offsets.MonthEnd(0)
data3 = data3.rename(columns={'dateff': 'datadate'})

#### Organizing & Undestanding the data 

In [ ]:
# Cleaning the data: Keeping only the rows that don't contain 'N.M.' or 'SD'

mask = (merged_data['splticrm'] != 'N.M.') & (merged_data['splticrm'] != 'SD') & (merged_data['splticrm'] != 'Suspended')
merged_data = merged_data[mask]

# Replacing Credit Risk with numeric values

splticrm_mapping = {
    "AAA": 1.0,
    "AA+": 2.0,
    "AA": 3.0,
    "AA-": 4.0,
    "A+": 5.0,
    "A": 6.0,
    "A-": 7.0,
    "BBB+": 8.0,
    "BBB": 9.0,
    "BBB-": 10.0,
    "BB+": 11.0,
    "BB": 12.0,
    "BB-": 13.0,
    "B+": 14.0,
    "B": 15.0,
    "B-": 16.0,
    "CCC+": 17.0,
    "CCC": 18.0,
    "CCC-": 19.0,
    "CC": 20.0,
    "C": 21.0,
    "D": 22.0
}

merged_data.loc[:, 'splticrm'] = merged_data['splticrm'].replace(splticrm_mapping)


In [ ]:
# Matching sector codes to sector names

sector_names = {
    10: '10 Energy',
    15: '15 Materials',
    20: '20 Industrials',
    25: '25 Consumer Discretionary',
    30: '30 Consumer Staples',
    35: '35 Health Care',
    40: '40 Financials',
    45: '45 Information Technology',
    50: '50 Communication Services',
    55: '55 Utilities',
    60: '60 Real Estate'
}

In [ ]:
monthly_returns = merged_data.groupby(['datadate', 'gsector'])['trt1m'].mean().reset_index()
monthly_returns = monthly_returns.rename(columns={'trt1m': 'monthly_return'})
pivot_table10 = monthly_returns.pivot_table(index='datadate', columns='gsector', values='monthly_return')
pivot_table10.reset_index(inplace=True)
pivot_table10['datadate'] = pd.to_datetime(pivot_table10['datadate'])
Data40 = pd.merge(pivot_table10, data3[['datadate', 'mktrf']], on='datadate', how='left')

# Getting the betas of each sector 
sector_betas = []

for sector_column in Data40.columns[1:-1]: 
    sector_number = sector_column  
    y = Data40[sector_column]  
    X = sm.add_constant(Data40['mktrf'])  
    model = sm.OLS(y, X).fit()  
    beta = model.params['mktrf']  
    sector_betas.append({'gsector': sector_number, 'Beta': beta})

    beta_df = pd.DataFrame(sector_betas)
beta_df

#### Sectors Portfolios 

In [ ]:
# Getting the excess returns 

merged_data['datadate'] = pd.to_datetime(merged_data['datadate'], format='%Y-%m-%d')
merged_data = merged_data.sort_values(by='datadate')

excess_returns = []

for index, row in merged_data.iterrows():
    date = row['datadate']
    
    matching_row = data3[data3['datadate'] == date]
    
    if not matching_row.empty:
        excess_return = row['trt1m'] - matching_row.iloc[0]['rf']
    else:
        excess_return =  row['trt1m'] 
        
    excess_returns.append(excess_return)

merged_data['excess_returns'] = excess_returns


In [ ]:
# Forming the Equal Weights Long-Short Portfolio for each Sector

merged_data1 = merged_data.copy()
merged_data1['signal'] = np.where(merged_data1['splticrm'] > 10, -1, 1)
merged_data1['portfolio_returns'] = merged_data1['signal'] * merged_data1['excess_returns']
long_short_portfolio = merged_data1.groupby(['datadate', 'gsector'])['portfolio_returns'].mean().reset_index()
long_short_portfolio.set_index('datadate', inplace=True)
long_short_portfolio['adjusted_portfolio_returns'] = long_short_portfolio['portfolio_returns'] + 1
long_short_portfolio['cumulative_returns'] = long_short_portfolio.groupby('gsector')['adjusted_portfolio_returns'].cumprod() - 1
long_short_portfolio.reset_index(inplace=True)
data3['cumulative_mktrf'] = (1 + data3['mktrf']).cumprod() - 1


# Plotting
fig = px.line(long_short_portfolio, x='datadate', y='cumulative_returns', color='gsector',
              labels={'datadate': 'Date', 'cumulative_returns': 'Cumulative Returns'},
              line_shape='linear')

fig.add_scatter(x=data3['datadate'],
                y=data3['cumulative_mktrf'],  
                line=dict(dash='dot'),  
                name='mktrf')
                
fig.update_layout(
    title='Graph 1: Sectors Portfolios Cumulative Returns',
    title_y = 0.95,
    title_x = 0.15,
    xaxis_title='Date',  
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(
        showline=True,
        showgrid=True,
        gridwidth=0.2,
        gridcolor='lightgray',
        showticklabels=True,
        linecolor='black'
    ),
    yaxis=dict(
        showline=True,
        showgrid=True,
        gridwidth=0.2,
        gridcolor='lightgray',
        showticklabels=True,
        linecolor='black',
        zeroline=True,  
        zerolinewidth=0.5,
        zerolinecolor='lightgray'
    ),
    plot_bgcolor='white',  
    paper_bgcolor='white', 
    legend_title='',
     legend=dict(
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=14)
    )
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

for code, name in sector_names.items():
    for trace in fig.data:
        if trace.name == str(code):
            trace.name = name

fig.update_layout(width=900, height=400)

fig.show()

In [ ]:
# Performance Measures - Annual return, Volatility and Sharpe Ratio - Full Period

import numpy as np

def calculate_annual_return(returns):
    geometric_mean_return = np.prod(1 + returns) ** (1 / len(returns)) - 1
    annual_return = (1 + geometric_mean_return) ** 12 - 1
    return annual_return


annualized_returns = long_short_portfolio.groupby('gsector')['portfolio_returns'].apply(calculate_annual_return)
performance_metrics = pd.DataFrame({'gsector': annualized_returns.index, 'Annualized Return': annualized_returns.values})

def calculate_annualized_volatility(returns):
    monthly_volatility = np.std(returns)
    annual_volatility = monthly_volatility * np.sqrt(12) 
    return annual_volatility

annualized_volatility = long_short_portfolio.groupby('gsector')['portfolio_returns'].apply(calculate_annualized_volatility)
volatility_df = annualized_volatility.reset_index()  

volatility_df.columns = ['gsector', 'Volatility']

performance_metrics = performance_metrics.merge(volatility_df, on='gsector', how='left')

performance_metrics['Sharpe Ratio'] = performance_metrics['Annualized Return'] / performance_metrics['Volatility']

mktrf_annual_return = calculate_annual_return(data3['mktrf'])

mktrf_annualized_volatility = calculate_annualized_volatility(data3['mktrf'])

mktrf_sharpe_ratio = mktrf_annual_return / mktrf_annualized_volatility

mktrf_stats = pd.DataFrame({'gsector': ['mktrf'], 'Annualized Return': [mktrf_annual_return], 'Volatility': [mktrf_annualized_volatility], 'Sharpe Ratio': [mktrf_sharpe_ratio]})

performance_metrics = pd.concat([performance_metrics, mktrf_stats], ignore_index=True)

performance_metrics.set_index('gsector', inplace=True)

performance_metrics



#### Creating the Long-only and Long-short portfolios with the best performing sectors 

In [ ]:
# Selecting the Best Performing Portfolios per month

long_short_portfolio_new = long_short_portfolio.copy()
start = pd.to_datetime('1988-05-01')

long_short_portfolio_new['sharpe_ratio'] = long_short_portfolio_new.groupby('gsector')['portfolio_returns'].transform(
    lambda x: x.expanding().mean() / x.expanding().std())
long_short_portfolio_new['cumulative_returns'] = long_short_portfolio_new.groupby('gsector')['adjusted_portfolio_returns'].transform(
    lambda x: x.expanding().apply(np.prod) - 1)

long_short_portfolio_new = long_short_portfolio_new[long_short_portfolio_new['datadate'] >= start]

long_short_portfolio_new['rank_sharpe'] = long_short_portfolio_new.groupby('datadate')['sharpe_ratio'].rank(ascending=False)
long_short_portfolio_new['rank_cum'] = long_short_portfolio_new.groupby('datadate')['cumulative_returns'].rank(ascending=False)

long_short_portfolio_new['final_rank'] = long_short_portfolio_new[['rank_sharpe', 'rank_cum']].mean(axis=1)

top3_sectors = long_short_portfolio_new.groupby('datadate').apply(lambda x: x.nsmallest(3, 'final_rank')).reset_index(drop=True)
top3_sectors = top3_sectors.dropna(subset=['final_rank'])

unique_combinations = top3_sectors[['datadate', 'gsector']]

merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
unique_combinations['datadate'] = pd.to_datetime(unique_combinations['datadate'])

filtered_data_new= merged_data[merged_data.set_index(['datadate', 'gsector']).index.isin(unique_combinations.set_index(['datadate', 'gsector']).index)]



# Long-short portfolio

filtered_data_new1 = filtered_data_new.copy()
filtered_data_new1['signal'] = np.where(filtered_data_new1['splticrm'] > 10, -1, 1)
filtered_data_new1['portfolio_returns'] = filtered_data_new1['signal'] * filtered_data_new1['excess_returns']
new_long_short = filtered_data_new1.groupby(['datadate'])['portfolio_returns'].mean().reset_index()
new_long_short.set_index('datadate', inplace=True)
new_long_short['adjusted_portfolio_returns'] = new_long_short['portfolio_returns'] + 1
new_long_short['cumulative_returns'] = new_long_short['adjusted_portfolio_returns'].cumprod() - 1
new_long_short.reset_index(inplace=True)


# Long portfolio
filtered_data_new2 = filtered_data_new.copy()
filtered_data_new2 = filtered_data_new2[filtered_data_new2['splticrm'] < 11]
long_portfolio = filtered_data_new2[['datadate', 'excess_returns']]
long_portfolio = long_portfolio.groupby(['datadate'])['excess_returns'].mean().reset_index()
long_portfolio['adjusted_portfolio_returns'] = long_portfolio['excess_returns'] + 1
long_portfolio['cumulative_returns'] = long_portfolio['adjusted_portfolio_returns'].cumprod() - 1

# Market 
data3 = data3[data3['datadate'] >= '1988-05-31']
data3['cumulative_mktrf'] = (1 + data3['mktrf']).cumprod() - 1

long_portfolio['datadate'] = pd.to_datetime(long_portfolio['datadate'])
new_long_short['datadate'] = pd.to_datetime(new_long_short['datadate'])
data3['datadate'] = pd.to_datetime(data3['datadate'])

long_portfolio['datadate'] = long_portfolio['datadate'].dt.strftime('%Y-%m-%d')
new_long_short['datadate'] = new_long_short['datadate'].dt.strftime('%Y-%m-%d')
data3['datadate'] = data3['datadate'].dt.strftime('%Y-%m-%d')

# Plotting
trace_long_portfolio = go.Scatter(x=long_portfolio['datadate'], y=long_portfolio['cumulative_returns'],
                                  name='Long Portfolio', line_shape='linear')

trace_long_short_portfolio = go.Scatter(x=new_long_short['datadate'], y=new_long_short['cumulative_returns'],
                                        name='Long-Short Portfolio', line_shape='linear')

trace_mktrf = go.Scatter(x=data3['datadate'], y=data3['cumulative_mktrf'],
                        name='Market Returns', line_shape='linear', line=dict(dash='dot'))

layout = go.Layout(
    title='Graph 2: Portfolios Cumulative Returns',
    title_y = 0.88,
    title_x = 0.2,
    title_font=dict(size=18),
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(
        title='Portfolio Cumulative Return',
        showline=True,
        showgrid=True,
        gridwidth=0.2,
        gridcolor='lightgray',
        showticklabels=True,
        linecolor='black',
        zeroline=True,  
        zerolinecolor='lightgray', 
        zerolinewidth=1 
    ),
    xaxis_linecolor='black',
    yaxis_linecolor='black',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=14)
    ),
)

fig = go.Figure(data=[trace_long_portfolio, trace_long_short_portfolio, trace_mktrf], layout=layout)

fig.add_shape(
    type="line",
    x0=long_portfolio['datadate'].min(),
    x1=long_portfolio['datadate'].min(),
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=900, height=400)

fig.show()


In [ ]:
# Performance Metrics - Full Period

data = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return': [
        calculate_annual_return(long_portfolio['excess_returns']),
        calculate_annual_return(new_long_short['portfolio_returns']),
        calculate_annual_return(data3['mktrf'])
    ],
    'Volatility': [
        calculate_annualized_volatility(long_portfolio['excess_returns']),
        calculate_annualized_volatility(new_long_short['portfolio_returns']),
        calculate_annualized_volatility(data3['mktrf'])
    ]
}

data['Sharpe Ratio'] = [
    data['Annual Return'][i] / data['Volatility'][i] for i in range(len(data['Annual Return']))
]

performance = pd.DataFrame(data)
performance

In [ ]:
# Dividing the Data on First and Second Halves - Performance Measuments

# Long-short

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Long

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

# Factors

data3.sort_values(by='datadate', inplace=True)

midpoint_data3 = len(data3) // 2

first_half_data3 = data3.iloc[:midpoint_data3]
second_half_data3 = data3.iloc[midpoint_data3:]

In [ ]:
# Performance Metrics - First Half

data_first_half = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return first_half': [
        calculate_annual_return(long_first_half['excess_returns']),
        calculate_annual_return(longshort_first_half['portfolio_returns']),
        calculate_annual_return(first_half_data3['mktrf'])
    ],
    'Volatility first_half': [
        calculate_annualized_volatility(long_first_half['excess_returns']),
        calculate_annualized_volatility(longshort_first_half['portfolio_returns']),
        calculate_annualized_volatility(first_half_data3['mktrf'])
    ]
}

# Calculate Sharpe Ratio
data_first_half['Sharpe Ratio first_half'] = [
    data_first_half['Annual Return first_half'][i] / data_first_half['Volatility first_half'][i] for i in range(len(data_first_half['Annual Return first_half']))
]

performance_first_half = pd.DataFrame(data_first_half)
performance_first_half

In [ ]:
# Performance Metrics - Second Half

data_second_half = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return second_half': [
        calculate_annual_return(long_second_half['excess_returns']),
        calculate_annual_return(longshort_second_half['portfolio_returns']),
        calculate_annual_return(second_half_data3['mktrf'])
    ],
    'Volatility second_half': [
        calculate_annualized_volatility(long_second_half['excess_returns']),
        calculate_annualized_volatility(longshort_second_half['portfolio_returns']),
        calculate_annualized_volatility(second_half_data3['mktrf'])
    ]
}

data_second_half['Sharpe Ratio second_half'] = [
    data_second_half['Annual Return second_half'][i] / data_second_half['Volatility second_half'][i] for i in range(len(data_second_half['Annual Return second_half']))
]

performance_second_half = pd.DataFrame(data_second_half)
performance_second_half

In [ ]:
# Organizing the data for CAPM

new_long_short = new_long_short.merge(data3[['datadate', 'mktrf']], on='datadate', how='left')
long_portfolio = long_portfolio.merge(data3[['datadate', 'mktrf']], on='datadate', how='left')

In [ ]:
# CAPM - Full period

results_CAPM = []

# Long Portfolio
y_long = long_portfolio['excess_returns']
X_long = long_portfolio[['mktrf']]
X_long = sm.add_constant(X_long)

model_long = sm.OLS(y_long, X_long).fit()

alpha_long = model_long.params['const']
beta_long = model_long.params['mktrf']
t_statistic_alpha_long = model_long.tvalues['const']
t_statistic_beta_long = model_long.tvalues['mktrf']

excess_returns_long = y_long
tracking_error_long = excess_returns_long.std()
ir_long = alpha_long / tracking_error_long

results_CAPM.append({
    'Portfolio': 'Long Portfolio',
    'Alpha (CAPM)': alpha_long,
    'T-Statistic (Alpha)': t_statistic_alpha_long,
    'Beta (CAPM)': beta_long,
    'T-Statistic (Beta)': t_statistic_beta_long,
    'Information Ratio (CAPM)': ir_long,
})

# Long-Short Portfolio
y_longshort = new_long_short['portfolio_returns']
X_longshort = new_long_short[['mktrf']]
X_longshort = sm.add_constant(X_longshort)

model_longshort = sm.OLS(y_longshort, X_longshort).fit()

alpha_longshort = model_longshort.params['const']
beta_longshort = model_longshort.params['mktrf']
t_statistic_alpha_longshort = model_longshort.tvalues['const']
t_statistic_beta_longshort = model_longshort.tvalues['mktrf']

excess_returns_longshort = y_longshort
tracking_error_longshort = excess_returns_longshort.std()
ir_longshort = alpha_longshort / tracking_error_longshort

results_CAPM.append({
    'Portfolio': 'Long-short Portfolio',
    'Alpha (CAPM)': alpha_longshort,
    'T-Statistic (Alpha)': t_statistic_alpha_longshort,
    'Beta (CAPM)': beta_longshort,
    'T-Statistic (Beta)': t_statistic_beta_longshort,
    'Information Ratio (CAPM)': ir_longshort,
})

results_CAPM_df = pd.DataFrame(results_CAPM)
results_CAPM_df



In [ ]:
# Splitting Long-short for CAPM

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Splitting Long for CAPM

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

In [ ]:
# CAPM in-sample & out-of-sample 

results_CAPM = []

# Long Portfolio
for period, half_data in enumerate([long_first_half, long_second_half]):
    y_long = half_data['excess_returns']
    X_long = half_data[['mktrf']]
    X_long = sm.add_constant(X_long)

    model_long = sm.OLS(y_long, X_long).fit()

    alpha_long = model_long.params['const']
    beta_long = model_long.params['mktrf']
    t_statistic_alpha_long = model_long.tvalues['const']
    t_statistic_beta_long = model_long.tvalues['mktrf']

    excess_returns_long = y_long
    tracking_error_long = excess_returns_long.std()
    ir_long = alpha_long / tracking_error_long

    results_CAPM.append({
        'Portfolio': 'Long Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (CAPM)': alpha_long,
        'T-Statistic (Alpha)': t_statistic_alpha_long,
        'Beta (CAPM)': beta_long,
        'T-Statistic (Beta)': t_statistic_beta_long,
        'Information Ratio (CAPM)': ir_long,
    })

# Long-Short Portfolio
for period, half_data in enumerate([longshort_first_half, longshort_second_half]):
    y_longshort = half_data['portfolio_returns']
    X_longshort = half_data[['mktrf']]
    X_longshort = sm.add_constant(X_longshort)

    model_longshort = sm.OLS(y_longshort, X_longshort).fit()

    alpha_longshort = model_longshort.params['const']
    beta_longshort = model_longshort.params['mktrf']
    t_statistic_alpha_longshort = model_longshort.tvalues['const']
    t_statistic_beta_longshort = model_longshort.tvalues['mktrf']

    excess_returns_longshort = y_longshort
    tracking_error_longshort = excess_returns_longshort.std()
    ir_longshort = alpha_longshort / tracking_error_longshort

    results_CAPM.append({
        'Portfolio': 'Long-short Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (CAPM)': alpha_longshort,
        'T-Statistic (Alpha)': t_statistic_alpha_longshort,
        'Beta (CAPM)': beta_longshort,
        'T-Statistic (Beta)': t_statistic_beta_longshort,
        'Information Ratio (CAPM)': ir_longshort,
    })

results_CAPM_df = pd.DataFrame(results_CAPM)
results_CAPM_df



In [ ]:
# Organizing the data for the FF3

new_long_short = new_long_short.merge(data3[['datadate', 'smb', 'hml']], on='datadate', how='left')
long_portfolio = long_portfolio.merge(data3[['datadate', 'smb', 'hml']], on='datadate', how='left')


In [ ]:
# FF3 - Full period

results_ff3 = []

# Long Portfolio
y_long = long_portfolio['excess_returns']
X_long = long_portfolio[['mktrf', 'smb', 'hml']]
X_long = sm.add_constant(X_long)

model_long = sm.OLS(y_long, X_long).fit()

alpha_long = model_long.params['const']
t_statistic_alpha_long = model_long.tvalues['const']

beta_mkt_long = model_long.params['mktrf']
t_statistic_mkt_long = model_long.tvalues['mktrf']

beta_smb_long = model_long.params['smb']
t_statistic_smb_long = model_long.tvalues['smb']

beta_hml_long = model_long.params['hml']
t_statistic_hml_long = model_long.tvalues['hml']

excess_returns_long = y_long
tracking_error_long = excess_returns_long.std()
ir_long = alpha_long / tracking_error_long

results_ff3.append({
    'Portfolio': 'Long Portfolio',
    'Alpha (FF3)': alpha_long,
    'T-Statistic Alpha': t_statistic_alpha_long,
    'Beta Mkt-Rf': beta_mkt_long,
    'T-Statistic Mkt-Rf': t_statistic_mkt_long,
    'Beta SMB': beta_smb_long,
    'T-Statistic SMB': t_statistic_smb_long,
    'Beta HML': beta_hml_long,
    'T-Statistic HML': t_statistic_hml_long,
    'Information Ratio (FF3)': ir_long
})

# Long-Short Portfolio
y_longshort = new_long_short['portfolio_returns']
X_longshort = new_long_short[['mktrf', 'smb', 'hml']]
X_longshort = sm.add_constant(X_longshort)

model_longshort = sm.OLS(y_longshort, X_longshort).fit()

alpha_longshort = model_longshort.params['const']
t_statistic_alpha_longshort = model_longshort.tvalues['const']

beta_mkt_longshort = model_longshort.params['mktrf']
t_statistic_mkt_longshort = model_longshort.tvalues['mktrf']

beta_smb_longshort = model_longshort.params['smb']
t_statistic_smb_longshort = model_longshort.tvalues['smb']

beta_hml_longshort = model_longshort.params['hml']
t_statistic_hml_longshort = model_longshort.tvalues['hml']

excess_returns_longshort = y_longshort
tracking_error_longshort = excess_returns_longshort.std()
ir_longshort = alpha_longshort / tracking_error_longshort

results_ff3.append({
    'Portfolio': 'Long-Short Portfolio',
    'Alpha (FF3)': alpha_longshort,
    'T-Statistic Alpha': t_statistic_alpha_longshort,
    'Beta Mkt-Rf': beta_mkt_longshort,
    'T-Statistic Mkt-Rf': t_statistic_mkt_longshort,
    'Beta SMB': beta_smb_longshort,
    'T-Statistic SMB': t_statistic_smb_longshort,
    'Beta HML': beta_hml_longshort,
    'T-Statistic HML': t_statistic_hml_longshort,
    'Information Ratio (FF3)': ir_longshort
})

results_ff3_df = pd.DataFrame(results_ff3)
results_ff3_df


In [ ]:
# Splitting Long-short for FF3

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Splitting Long for FF3

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

In [ ]:
# FF3 in-sample & out-of-sample 

results_ff3 = []

# Long Portfolio
for period, half_data in enumerate([long_first_half, long_second_half]):
    y_long = half_data['excess_returns']
    X_long = half_data[['mktrf', 'smb', 'hml']]
    X_long = sm.add_constant(X_long)

    model_long = sm.OLS(y_long, X_long).fit()

    alpha_long = model_long.params['const']
    t_statistic_alpha_long = model_long.tvalues['const']

    beta_mkt_long = model_long.params['mktrf']
    t_statistic_mkt_long = model_long.tvalues['mktrf']

    beta_smb_long = model_long.params['smb']
    t_statistic_smb_long = model_long.tvalues['smb']

    beta_hml_long = model_long.params['hml']
    t_statistic_hml_long = model_long.tvalues['hml']

    excess_returns_long = y_long
    tracking_error_long = excess_returns_long.std()
    ir_long = alpha_long / tracking_error_long

    results_ff3.append({
        'Portfolio': 'Long Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (FF3)': alpha_long,
        'T-Statistic Alpha': t_statistic_alpha_long,
        'Beta Mkt-Rf': beta_mkt_long,
        'T-Statistic Mkt-Rf': t_statistic_mkt_long,
        'Beta SMB': beta_smb_long,
        'T-Statistic SMB': t_statistic_smb_long,
        'Beta HML': beta_hml_long,
        'T-Statistic HML': t_statistic_hml_long,
        'Information Ratio (FF3)': ir_long
    })

# Long-Short Portfolio
for period, half_data in enumerate([longshort_first_half, longshort_second_half]):
    y_longshort = half_data['portfolio_returns']
    X_longshort = half_data[['mktrf', 'smb', 'hml']]
    X_longshort = sm.add_constant(X_longshort)

    model_longshort = sm.OLS(y_longshort, X_longshort).fit()

    alpha_longshort = model_longshort.params['const']
    t_statistic_alpha_longshort = model_longshort.tvalues['const']

    beta_mkt_longshort = model_longshort.params['mktrf']
    t_statistic_mkt_longshort = model_longshort.tvalues['mktrf']

    beta_smb_longshort = model_longshort.params['smb']
    t_statistic_smb_longshort = model_longshort.tvalues['smb']

    beta_hml_longshort = model_longshort.params['hml']
    t_statistic_hml_longshort = model_longshort.tvalues['hml']

    excess_returns_longshort = y_longshort
    tracking_error_longshort = excess_returns_longshort.std()
    ir_longshort = alpha_longshort / tracking_error_longshort

    results_ff3.append({
        'Portfolio': 'Long-short Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (FF3)': alpha_longshort,
        'T-Statistic Alpha': t_statistic_alpha_longshort,
        'Beta Mkt-Rf': beta_mkt_longshort,
        'T-Statistic Mkt-Rf': t_statistic_mkt_longshort,
        'Beta SMB': beta_smb_longshort,
        'T-Statistic SMB': t_statistic_smb_longshort,
        'Beta HML': beta_hml_longshort,
        'T-Statistic HML': t_statistic_hml_longshort,
        'Information Ratio (FF3)': ir_longshort
    })

results_ff3_df = pd.DataFrame(results_ff3)
results_ff3_df


In [ ]:
# Calculating the drawdowns
def calculate_drawdowns(wealth_index):
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    return drawdowns

drawdowns_long = calculate_drawdowns(long_portfolio['cumulative_returns']+1)
drawdowns_long_short = calculate_drawdowns(new_long_short['cumulative_returns']+1)

# Plotiing
fig = go.Figure()

fig.add_trace(go.Scatter(x=long_portfolio['datadate'], y=drawdowns_long, mode='lines', name='Long Portfolio'))
fig.add_trace(go.Scatter(x=new_long_short['datadate'], y=drawdowns_long_short, mode='lines', name='Long-Short Portfolio'))

fig.update_layout(
    title='Graph 3: Portfolios Drawdowns',
    title_y = 0.8,
    title_x = 0.1,  
    title_yanchor ='bottom',
    title_font=dict(size=18),
    xaxis_title='Date',
    yaxis_title='Drawdown',
    paper_bgcolor='white',  
    plot_bgcolor='white',   
    xaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5),  
    yaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5, zeroline=True, zerolinecolor='lightgray', zerolinewidth=0.5), 
    legend=dict(x=0.01, y=0.01, traceorder='normal', bgcolor='white', bordercolor='black', borderwidth=0.2, font=dict(size=13.5)), 
    height=350, 
    width=1000,   
)

fig.show()
